In [6]:
import cv2

In [7]:
#COCO model
# 440000 caffe
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

# POSE_PAIRS = [['Nose', 'Neck'], ['Neck', 'LShoulder'], ['Neck', 'RShoulder'], ['LShoulder', 'LElbow'], ['RShoulder', 'RElbow'], ['LElbow', 'LWrist'], ['RElbow', 'RWrist'], ['Neck', 'LHip'], ['Neck', 'RHip'], ["LHip", 'LKnee'], ["RHip", "RKnee"], ['LKnee', 'LAnkle'], ['RKnee', 'RAnkle']] #use only 14pairs
POSE_PAIRS = [[0, 1], [0, 14], [0, 15], [1, 2], [1, 5], [1, 8], [1, 11], [2, 3], [3, 4],
                   [5, 6], [6, 7], [8, 9], [9, 10], [12, 13], [11, 12], [14, 16], [15, 17]]


In [9]:
proto_file = 'E:\\2023\\2023_1_1\\rne\openpose\models\pose\coco\pose_deploy_linevec.prototxt'
weights_file = 'E:\\2023\\2023_1_1\\rne\openpose\models\pose\coco\pose_iter_440000.caffemodel'

In [46]:
def output_keypoints(frame, proto_file, weights_file):
    global points
    points = []

    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)

    frame_width = frame[0]
    frame_height = frame[1]

    net.setInput(cv2.dnn.blobFromImage(frame, 1.0 / 255, (frame_width, frame_height), (0, 0, 0), swapRB=False, crop=False))
    out = net.forward()
    out = out[:, :19, :, :] #네트워크의 정해진 ouput이 있어서 마음대로 벡터 연결시키면 안됨. 우선 18 keypoints 얻고 필요시 제거

    thres = 0.5
    for i in range(len(BODY_PARTS)):
        prob_map = out[0, i, :, :]

        # 최소값, 최대값, 최소값 위치, 최대값 위치
        min_val, conf, min_loc, point = cv2.minMaxLoc(prob_map) #global maximum을 찾아서 한명만 추정됨. local은 그냥 두 명 이상을 yolo로 찾아서 대체

        x = int((frame_width * point[0]) / frame_width)
        y = int((frame_height * point[1]) / frame_height)
        if conf > thres:
            points.append((x, y))
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
        else: points.append(None)
    return frame

In [ ]:
# for pair in POSE_PAIRS:
#         partFrom = pair[0]
#         partTo = pair[1]

In [11]:
def get_joints(frame):
    circled_frame = output_keypoints(frame, proto_file, weights_file)
    return points



In [ ]:
def draw_bodypose(canvas, candidate, subset):
    stickwidth = 4
    limbSeq = [[2, 3], [2, 6], [3, 4], [4, 5], [6, 7], [7, 8], [2, 9], [9, 10], \
               [10, 11], [2, 12], [12, 13], [13, 14], [2, 1], [1, 15], [15, 17], \
               [1, 16], [16, 18], [3, 17], [6, 18]]

